***
# Sequential Circuits in Chisel


## Prof. Jan Reineke, Saarland University
### reineke@cs.uni-saarland.de

### These slides are not meant to be consumed "stand-alone" but rather as a companion to the corresponding "theory" slides.

### Partially based on slides by Scott Beamer, University of California, Santa Cruz (https://github.com/agile-hw/lectures)

## Plan for Today

* Registers 
* Memories 
* Finite-state machines

## Loading The Chisel Library Into a Notebook

In [ ]:
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))


In [ ]:
import ammonite.repl._

import chisel3._
import chisel3.util._ 
import chiseltest._
import chiseltest.RawTester.test

In [ ]:
repl.newCompiler
repl.load.exec(os.Path(s"${System.getProperty("user.dir")}/../resource/deps.scala"))

## Registers

* Explicitly declare a register with `Reg(type)`
  * Contrast from Verilog where registers are synthesized
  * `type` can be any Chisel data type, e.g. `Boolean` or `UInt`, but also vectors
* A register is simply another block, no special semantics for time
  * Simply need to connect its input and its output
* `clock` and `reset` are implicit
* Not covered in this course: functionality for using other clocks ([multi-clock](https://www.chisel-lang.org/chisel3/docs/explanations/multi-clock.html) and [asynchronous reset](https://www.chisel-lang.org/chisel3/docs/explanations/reset.html))

<img src="images/reg.svg" alt="register schematic" style="width:100%;" align="center"/>

## Flavors of `Reg`

### [Set Initial Value](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/RegInit$.html) - `RegInit(init)`
* Value applied synchronously when `reset` true

### [Attach Input](https://javadoc.io/static/edu.berkeley.cs/chisel3_2.13/3.6.0/chisel3/RegNext$.html) - `RegNext(next, init)`
* Like `RegInit`, but directly connects input of register

### [Enable](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/RegEnable$.html) - `RegEnable(next, init, en)`
* Like `RegNext`, but with Write enable for when to update

In [ ]:
class RegLand extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val en  = Input(Bool())
        val out = Output(Bool())
    })
    val r = Reg(Bool())
//     val r = RegInit(0.B)
    r := io.in
    io.out := r
//     io.out := RegNext(io.in, 0.B)
//     io.out := RegEnable(io.in, 0.B, io.en)
}
visualize(() => new RegLand)

## *Example*: Counter (done manually)
Note this counter's behaves differently than the ``theory'' slides counter:
* Increment value when `en` is true
* Wrap around at `maxVal`
* Reset to `0`
* No ability to load particular value

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = Reg(UInt(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    val applyEn = Mux(io.en, nextVal, count)
    count := Mux(reset.asBool, 0.U, applyEn)
    io.out := count
}
visualize(() => new MyCounter(15))

## *Example*: Counter (using RegInit)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    count := Mux(io.en, nextVal, count)
    io.out := count
}
visualize(() => new MyCounter(15))

## *Example*: Counter (using when)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}
visualize(() => new MyCounter(15))

## Example: Counter (using RegEnable, too dense?)

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt(log2Ceil(maxVal+1).W))
    })
    io.out := RegEnable(Mux(io.out < maxVal.U, io.out + 1.U, 0.U), 0.U, io.en)
}
visualize(() => new MyCounter(15))

## Testing MyCounter

In [ ]:
test(new MyCounter(3)) { c =>
    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(1.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(2.U)
    c.clock.step()

    c.io.en.poke(0.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()
    println("Success!")
}

## $n$-Bit Decoder: Abstract Class and Tests

In [ ]:
abstract class Decoder(noInputs: Int) extends Module {
  val noOutputs = 1 << noInputs
  val io = IO(new Bundle {
    val in = Input(UInt(noInputs.W))
    val out = Output(UInt(noOutputs.W))
  })
}

  def randomDecoderTest(
    decoderGen: => Decoder,
    noInputs: Int,
    numberOfTests: Int
  ) = {
    test(decoderGen) { c =>
      val noOutputs = BigInt(1) << noInputs
      val rand = new Random
      for (_ <- 1 to numberOfTests) {
        setRandomInput(c.io, rand)
        val input = c.io.in.peekInt().toInt
        assert(c.io.out.peek().litValue == (BigInt(1) << input))
      }
    }
  }

## $n$-Bit Decoder


In [ ]:
class RecursiveDecoder(noInputs: Int) extends Decoder(noInputs) {
  if (noInputs == 1) {
    io.out := Cat(io.in, ~io.in)
  } else {
    val internalDecoder = Module(new RecursiveDecoder(noInputs - 1))
    internalDecoder.io.in := io.in(noInputs - 2, 0)
    
    val lowerOut = internalDecoder.io.out & Fill(noOutputs/2, ~io.in(noInputs - 1))
    val upperOut = internalDecoder.io.out & Fill(noOutputs/2, io.in(noInputs - 1))
    io.out := Cat(upperOut, lowerOut) 
  }
}

In [ ]:
randomDecoderTest(new RecursiveDecoder(10), noInputs = 10, numberOfTests = 10000)

## A Simpler $n$-Bit Decoder
... relying on built-in Chisel features:

In [ ]:
class SimpleDecoder(noInputs: Int) extends Decoder(noInputs) {
    io.out := 1.U << io.in
}

In [ ]:
randomDecoderTest(new SimpleDecoder(10), noInputs = 10, numberOfTests = 10000)

## Memories in Chisel

Cannot directly express latches in Chisel!

Thus, cannot build SRAM from ``theory'' slides from scratch.

However, Chisel has built-in support for memories.

## *Excursion:* Chisel `Vec`

* Chisel collection construct, two uses:
  * _Dynamic select_ in hardware (muxes) $\neq$ Scala collection during elaboration
  * Parameterize number of ports of a module
* `Vec(num_elements, type)`

* Use `Reg` of `Vec` for state
  * `Vec` of `Reg` not possible

```scala
Reg(Vec(num_elements, type))
```
* Can also use with `Wire`

```scala
Wire(Vec(num_elements, type))
```

## *Excursion:* Chisel `Vec` -  Multiplexer

In [ ]:
class MyMuxN(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))  // parameterize number of input ports
        val sel = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(w.W))
    })
    io.out := io.in(io.sel) //dynamic select
}
visualize(() => new MyMuxN(4,1))

## *Excursion*: Chisel `Vec` - Linear Reducer

In [ ]:
class LinearReducer(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
// like val in  = Input(UInt((n*w).W))
        val out = Output(UInt(w.W))
    })
    require(n > 0)
    var totalSoFar = io.in(0)
    for (i <- 1 until n)
        totalSoFar = io.in(i) + totalSoFar
    io.out := totalSoFar
}
visualize(() => new LinearReducer(4,2))

<img src="images/reducer.svg" alt="reducer schematic" style="width:65%;" align="right"/>

## *Excursion*: Chisel `Vec` - Logarithmic Reducer

In [ ]:
class LogarithmicReducer(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
        val out = Output(UInt(w.W))
    })
    require(n > 0)
    if (n == 1) {
        io.out := io.in(0)
    } else {
        val half = n/2
        val left = Module(new LogarithmicReducer(half, w))
        val right = Module(new LogarithmicReducer(n - half, w))
        left.io.in := io.in.take(half)
        right.io.in := io.in.drop(half)
        io.out := left.io.out + right.io.out
    }
}
visualize(() => new LogarithmicReducer(4,2))
visualizeHierarchy(() => new LogarithmicReducer(4,2))

## Memories in Chisel - Read-Only Memory (ROM) with `VecInit`
* `VecInit` will create a `Wire` with its argument
* Can also use `VecInit` to initialize registers (with `RegInit`)

In [ ]:
class SineTable(amplitude: Double, n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(log2Ceil(n).W))
        val out = Output(SInt(32.W))
    })

    val times = (0 until n).map(i => (i*2*math.Pi)/n.toDouble)
    val inits = times.map(t => math.round(amplitude * math.sin(t)).asSInt(32.W))
    val table = VecInit(inits)
    io.out := table(io.in)
}

In [ ]:
visualize(() => new SineTable(5, 10))

## Let's Play with `SineTable`

In [ ]:
val samples = 20; val amplitude = 20
test(new SineTable(amplitude, samples)) { c =>
    for (i <- 0 until samples) {
        c.io.in.poke(i.U)
        val out = c.io.out.peekInt().toInt
        println("*" * (out+amplitude+1))
    }
}

## Chisel `Mem`

* Construct for [memory](https://www.chisel-lang.org/chisel3/docs/explanations/memories.html) (dynamically addressable & **mutable**)
* Backend will choose appropriate implementation technology
* Default (Mem): _combinational read_ (0 cycle delay), _synchronous write_ (1 cycle delay)
  * Can tweak delay parameters
  * `SyncReadMem` and `SRAM` have 1 cycle read delay
* Memory ports can be declared implicitly or explicitly
* Also has support for write masks

## *Example:* Register File (2R, 1W) - Implicit Ports

In [ ]:
class RegFile(regs: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(log2Ceil(regs).W))
        val r1addr = Input(UInt(log2Ceil(regs).W))
        val w0addr = Input(UInt(log2Ceil(regs).W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(bitWidth.W))
        val r0out =  Output(UInt(bitWidth.W))
        val r1out =  Output(UInt(bitWidth.W))
    })
//     val regs = Mem(regs, UInt(bitWidth.W))
    val registers = Reg(Vec(regs, UInt(bitWidth.W)))
    io.r0out := registers(io.r0addr)
    io.r1out := registers(io.r1addr)
    when(io.w0en) {
        registers(io.w0addr) := io.w0data
    }
}

In [ ]:
visualize(() => new RegFile(32, 64))

## *Example:* Register File (2R, 1W) - Explicit Ports

In [ ]:
class RegFile(regs: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(log2Ceil(regs).W))
        val r1addr = Input(UInt(log2Ceil(regs).W))
        val w0addr = Input(UInt(log2Ceil(regs).W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(bitWidth.W))
        val r0out =  Output(UInt(bitWidth.W))
        val r1out =  Output(UInt(bitWidth.W))
    })
    val registers = Mem(regs, UInt(bitWidth.W))
    io.r0out := registers.read(io.r0addr)
    io.r1out := registers.read(io.r1addr)
    when(io.w0en) {
        registers(io.w0addr) := io.w0data
    }
}

## *Example:* Register File (NR, 1W) - Parameterized Number of Ports

In [ ]:
class RegFileParameterized(readPorts: Int, regs: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val raddr = Input(Vec(readPorts, UInt(log2Ceil(regs).W)))
        val w0addr = Input(UInt(log2Ceil(regs).W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(bitWidth.W))
        val rout =  Output(Vec(readPorts, UInt(bitWidth.W)))
    })
    val registers = Mem(regs, UInt(bitWidth.W))
    for (i <- 0 until readPorts) {
        io.rout(i) := registers.read(io.raddr(i))
        // io.rout(i) := registers(io.raddr(i))  //equivalent
    }
    when(io.w0en) {
        registers(io.w0addr) := io.w0data
    }
}

In [ ]:
visualize(() => new RegFileParameterized(4, 32, 64))

## When to Use Scala or Chisel Collection?

* You will want to use a Scala collection (probably `Seq`) most often
    * In a generator, want to instantiate N things
    * You may need to address which thing you want, but you are accessing it during hardware generation time
* You will need Chisel collections (`Vec`, `Mem`) when ...
    * You want the generated hardware to dynamically address components (put muxes into actual hardware)
    * You definitely want a memory (e.g. `Mem`, `SyncReadMem`)

## Vending Machine in Chisel

Let's implement the Vending Machine in Chisel: 

In [ ]:
 class VendingMachine extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(2.W))      //00=coin, 01=return, 10=option 1, 11=option 2
        val y = Output(UInt(2.W))     //00=no output, 01=money, 10=drink 1, 11=drink 2
    })
    val state = RegInit(0.B)
    state := !io.x(0) && !io.x(1)
    io.y := Cat(io.x(1) && state, io.x(0) && state)
    io.y := io.x & Fill(2, state)
}

In [ ]:
visualize(() => new VendingMachine)

## Let's Test the Vending Machine

Test needs access to state of vending machine. However, only inputs and outputs are exposed to the tester by default.

Can use (experimental) `expose` feature:

In [ ]:
import chiseltest.experimental.{expose}

class VendingMachineWrapper extends VendingMachine {
    val exposedState = expose(state)     //accessible to tester
}

## Testing the Vending Machine

In [ ]:
test(new VendingMachineWrapper) { c =>
    c.io.x.poke("b00".U)        //00 = coin
    c.io.y.expect("b00".U)      //00 = no output
    c.clock.step()
    c.exposedState.expect(1.B)  //state = 1 = full (coin inserted)

    c.io.x.poke("b01".U)        //01 = return
    c.io.y.expect("b01".U)      //01 = money
    c.clock.step()
    c.exposedState.expect(0.B)  //state = 0 = empty (no coin inserted)

    c.io.x.poke("b10".U)        //10 = option 1
    c.io.y.expect("b00".U)      //00 = no output, since no coin inserted
    c.clock.step()

    c.io.x.poke("b00".U)        //00 = coin
    c.clock.step()
    c.exposedState.expect(1.B)  //state = 1 = full (coin inserted)

    c.io.x.poke("b10".U)        //10 = option 1
    c.io.y.expect("b10".U)      //10 = drink 1
    c.clock.step()

    c.io.x.poke("b00".U)        //00 = coin
    c.clock.step()
    c.io.x.poke("b11".U)        //11 = option 2
    c.io.y.expect("b11".U)      //11 = drink 2   

    c.clock.step()
    c.exposedState.expect(0.B)  //state = 0 = empty (no coin inserted)  
    println("Success!")
}

## Vending Machine in Chisel - Discussion

Our implementation above is very "low-level" and hard to *read*, *test*, and *maintain*.

Let's implement a more readable version using Enums!

## Enums in Chisel (`ChiselEnum`)

* [`ChiselEnum`](https://javadoc.io/static/edu.berkeley.cs/chisel3_2.13/3.6.0/chisel3/ChiselEnum.html) provides [enumerations](https://www.chisel-lang.org/docs/explanations/chisel-enum) by assigning them `UInt`s (`Enumeration` is Scala)
* Helpful for putting human-sensical names to distinct values
* Example use cases
  * Naming states in a state machine
  * Labeling mux way selection options

In [ ]:
object DemoEnum extends ChiselEnum {
  val nameA, nameB, nameC = Value
  val nameD = Value(5.U)
  val anotherName = Value
}

println(DemoEnum.nameA.litValue, DemoEnum.nameB.litValue, DemoEnum.nameC.litValue, DemoEnum.nameD.litValue, DemoEnum.anotherName.litValue)

## Vending Machine Using Enums

First, let's define appropriate ChiselEnums for inputs, outputs, and internal state:

In [ ]:
object VMInputs extends ChiselEnum {
    val coin, ret, option1, option2 = Value
}
object VMOutputs extends ChiselEnum {
    val noOutput, money, drink1, drink2 = Value
}
object VMState extends ChiselEnum {
    val empty, full = Value
}

## Abstract Vending Machine Class

In [ ]:
abstract class VendingMachineWithEnums extends Module {
    val io = IO(new Bundle {
        val input = Input(VMInputs())
        val output = Output(VMOutputs())
    })
    val state = RegInit(VMState.empty)
} 

## Vending Machine Using Enums

In [ ]:
class NicerVendingMachine extends VendingMachineWithEnums {    
    when (state === VMState.empty) {
        when (io.input === VMInputs.coin) {
            state := VMState.full
        }
        io.output := VMOutputs.noOutput
    } .otherwise {
        when (io.input === VMInputs.ret) {
            io.output := VMOutputs.money
        } .elsewhen (io.input === VMInputs.option1) {
            io.output := VMOutputs.drink1
        } .elsewhen (io.input === VMInputs.option2) {
            io.output := VMOutputs.drink2
        } .otherwise {
            io.output := VMOutputs.noOutput
        }
        state := VMState.empty
    }
}

## Testing Vending Machines
We will build another version of the Vending Machine in a bit.

So let's create a generic wrapper to expose the internal state:


In [ ]:
class NicerVendingMachineWrapper(toWrap: => VendingMachineWithEnums) extends VendingMachineWithEnums {
    val vm = Module(toWrap)
    vm.io.input := io.input
    io.output := vm.io.output
    val expstate = expose(vm.state)
}

## Actually Testing the Vending Machine with Enums

In [ ]:
def testVendingMachineWithEnums(vm: => VendingMachineWithEnums) = {
    test (new NicerVendingMachineWrapper(vm)) { c =>
        c.io.input.poke(VMInputs.coin)        
        c.io.output.expect(VMOutputs.noOutput)      
        c.clock.step()
        c.expstate.expect(VMState.full)  

        c.io.input.poke(VMInputs.ret)        
        c.io.output.expect(VMOutputs.money)      
        c.clock.step()
        c.expstate.expect(VMState.empty)  

        c.io.input.poke(VMInputs.option1)        
        c.io.output.expect(VMOutputs.noOutput)      
        c.clock.step()

        c.io.input.poke(VMInputs.coin)        
        c.clock.step()
        c.expstate.expect(VMState.full)  

        c.io.input.poke(VMInputs.option1)        
        c.io.output.expect(VMOutputs.drink1)   
        c.clock.step()

        c.io.input.poke(VMInputs.coin)       
        c.clock.step()
        c.io.input.poke(VMInputs.option2)       
        c.io.output.expect(VMOutputs.drink2)     

        c.clock.step()
        c.expstate.expect(VMState.empty)  
        println("Success!")
    }
}

testVendingMachineWithEnums(new NicerVendingMachine)


## An Alternative Implementation Using `switch`

In [ ]:
class NicerVendingMachineWithSwitch extends VendingMachineWithEnums {
    io.output := VMOutputs.noOutput     //default
    switch(state) {
        is (VMState.empty) {
            when (io.input === VMInputs.coin) {
                state := VMState.full
            }
            io.output := VMOutputs.noOutput
        }
        is (VMState.full) {
            switch(io.input) {
                is (VMInputs.ret) {
                    io.output := VMOutputs.money
                }
                is (VMInputs.option1) {
                    io.output := VMOutputs.drink1
                }
                is (VMInputs.option2) {
                    io.output := VMOutputs.drink2
                }
                is (VMInputs.coin) {
                    state := VMState.full
                }
            }
            state := VMState.empty    
        }
    }
}

## ... And Its Test

In [ ]:
testVendingMachineWithEnums(new NicerVendingMachineWithSwitch)